# Galaxy Selector MOC

**Compared to the "Simple Galaxy Selector, this works with Multi-Ordering-Sky-Maps**


***NOTE (this note may be wrong, just copied from old notebook)***: Before anything can happen, download the GW event localization map. Open the terminal and type:
"curl -O https://gracedb.ligo.org/api/superevents/sid/files/bayestar.fits,0"
with sid = superevent ID (could be incorporated into this notebook)

Select the galaxies to observe by WWFI in this simple manner:
- get the LIGO event data
- read the LIGO event data into the notebook, extract some information
- read the DESI database into the notebook, "clean" the data
- get the 90% credible regions for the event, add them as a column to the data
- only keep data within the 90% region
- calculate all the luminosities (& more)
- rank them by luminosity (for now just print the TARGETID), 3D/2D localization and with a luminosity-distance 2D dependant counterpart likelihood

Sources: 
- https://iopscience.iop.org/article/10.3847/0067-0049/226/1/10
- https://emfollow.docs.ligo.org/userguide/tutorial/skymaps.html
- https://emfollow.docs.ligo.org/userguide/tutorial/multiorder_skymaps.html
- https://arxiv.org/pdf/1710.05452.pdf

## Import all libraries, load GW data and DESI data and extract basic info

### imports, GW data loading

In [113]:
import numpy as np
from scipy.stats import norm
import scipy as sc

from astropy.io import fits
from astropy.table import Table, hstack
from astropy import table

import healpy as hp
import astropy_healpix as ah

from astropy.table import QTable

import psycopg2

from scipy.stats import norm


from astropy.cosmology import Planck18, z_at_value
from astropy.coordinates import Distance, SkyCoord
from astropy import units as u

import astropy.constants as asc

import matplotlib.pyplot as plt

import gasel as gs

Specifiy the path where the data is located and then load the data (this loads everything, just have a look)

In [114]:
path = "/global/homes/j/jgassert/ligo-desi-gw-follow-up/Galaxy Selection/Files/S200129mbayestar.multiorder.fits,1"
skymap = QTable.read(path)

skymap[:5]

UNIQ,PROBDENSITY,DISTMU,DISTSIGMA,DISTNORM
,1 / sr,Mpc,Mpc,1 / Mpc2
int64,float64,float64,float64,float64
1024,1.5324656455152846e-49,714.1819585125542,161.80898167011566,1.8648435769602432e-06
1025,5.385644463361342e-50,721.1167587942227,171.98003188197197,1.8195498736811365e-06
1026,1.507013849287337e-56,512.6497643516975,199.2304182183156,3.3061639172754294e-06
1027,9.15663190324263e-58,624.4690102361741,203.60306281109953,2.3179848195476794e-06
1028,1.0313363324845354e-49,674.1376225795897,201.22336091582892,2.0204049691069628e-06


### Get basic values from GW event

We now extract some basic information  from the healpix data:
- ipix_max: pixel with highest likelihood
- npix: total number of pixels
- ra, dec: ra and dec of the pixel with the highest likelihood

Then we open the full fits file and extract some more basic info from the header:
- dist_mean: mean distance of the GW event
- dist_std: error of the distance

In [115]:
# most probable sky location
i = np.argmax(skymap["PROBDENSITY"])
uniq = skymap[i]["UNIQ"]
print("UNIQ ID of highest prob sky localization: ", uniq)

UNIQ ID of highest prob sky localization:  29382844


In [116]:
# calculate the most probable pixel, convert to RA and dec
level_max, ipix_max = ah.uniq_to_level_ipix(uniq)
nside = ah.level_to_nside(level_max)
ra, dec = ah.healpix_to_lonlat(ipix_max, nside, order = "nested")
print("Highest probability at (RA, dec): ", ra.deg, dec.deg, "with prob [1/deg^2] of: ", skymap[i]['PROBDENSITY'].to_value(u.deg**-2), "and UNIQ ID: ", skymap[i]["UNIQ"])

Highest probability at (RA, dec):  318.33984375 4.574345562095717 with prob [1/deg^2] of:  0.07437342287566126 and UNIQ ID:  29382844


Now open the fits file and read basics info from the header

In [117]:
fits_gw = fits.open(path)
header = fits_gw[1].header
dist_mean = header["DISTMEAN"]
dist_std = header["DISTSTD"]
print("The dist_mean and dist_std values: ", dist_mean*u.Mpc, dist_std*u.Mpc)

The dist_mean and dist_std values:  908.4386811451134 Mpc 202.2234141038336 Mpc


In [118]:
# these values (924, 188Mpc) are close to z= 0.19, 0.03

### Get DESI data

Establish a connection to the DESI database and load the daily data.

In [119]:
try:
    db = psycopg2.connect(host='decatdb.lbl.gov', database='desidb', user='desi', password = "5kFibers!", port="5432")
    cursor = db.cursor()
except (Exception, psycopg2.Error) as error:
    print(error)

cursor = db.cursor()

In [ ]:
redux = 'daily'
query = 'SELECT f.targetid,f.target_ra,f.target_dec,c.tileid,c.night,r.z,r.zerr,r.zwarn,r.deltachi2,f.flux_z,f.bgs_target,f.ebv, f.sersic, f.mws_target, c.filename\n' \
                    f'FROM {redux}.tiles_fibermap f\n' \
                    f'INNER JOIN {redux}.cumulative_tiles c ON f.cumultile_id=c.id\n' \
                    f'INNER JOIN {redux}.tiles_redshifts r ON r.cumultile_id=c.id AND r.targetid=f.targetid\n' \
                    f'WHERE q3c_radial_query( f.target_ra, f.target_dec, {ra.deg}, {dec.deg}, 50);'

cursor.execute(query)
rows = cursor.fetchall()

## Do all the calculations, selections,...

- turn the daily DESI data into a Table
- do some basic data selection stuff: only keep good + positive redshifts, only positive values of z-band flux and eliminate duplicates (right now simply takes the first entry)
- add the ipix (healpix) pixel value for each target
- calculate the probability for each target
- add these values to the Table ("PROB"), select only targets within the 90% credible region (i.e. with a certainty of 90% the GW event is inside this region)
- calculate the distances and its errors from the redshift, add to the data table
- calculate the 3D probability just like here: https://iopscience.iop.org/article/10.3847/0067-0049/226/1/10 (§4) and add these values to the table
- sort the table by its 3D probability (descending)
- calculate absolute and apparent magnitude, luminosity and add to table

The final step is to include the calculation of the most likely host galaxy from https://arxiv.org/pdf/1710.05452.pdf (GW170817); these values are then also added to the table as "P_GAL"

### Create DESI data Table and data clean up

In [ ]:
if rows:
    data = Table(list(map(list, zip(*rows))),
                             names=['TARGETID', 'TARGET_RA', 'TARGET_DEC', 'TILEID', 'NIGHT', 'Z', 'ZERR', 'ZWARN', 'DELTACHI2', 'FLUX_Z', 'BGS_TARGET', 'EBV', 'SERSIC', 'MWS_TARGET','FILENAME'])
data[:5:-1] 

In [ ]:
data = data[data['ZWARN']==0]
data = data[data['Z']>=0]
data = data[data['FLUX_Z']>0]
data = table.unique(data, keys = "TARGETID")

In [ ]:
print(f"We have {len(data)} objects in the full catalog") 

### Calculate the healpix id for every object in the catalog and its probability; append this data to the Table

In [ ]:
max_level = 29 # highest possible HEALPix resolution that can be represented in a 64-bit signed integer
max_nside = ah.level_to_nside(max_level)
level, ipix = ah.uniq_to_level_ipix(skymap["UNIQ"])

index = ipix*(2**(max_level-level))**2

sorter = np.argsort(index)

# this is the NESTED pixel index of the target sky location, NOT THE UNIQ ID
match_ipix = ah.lonlat_to_healpix(data["TARGET_RA"]*u.deg, data["TARGET_DEC"]*u.deg, max_nside, order='nested')

# here we get the pixel index (i.e. where the entry is in the table, also NOT THE UNIQ ID) of each target
sorter_i = sorter[np.searchsorted(index, match_ipix, side='right', sorter=sorter) - 1]
probdensity = skymap[sorter_i]['PROBDENSITY'].to_value(u.deg**-2)

# now we calculate the UNIQ ID for all our targets:

uniq = skymap["UNIQ"][sorter_i]
#to make things easier in future, I will now also append the UNIQ pixels DISTMU, DISTSIGMA and DISTNORM values. However, be careful, since these DO NOT CORRESPOND THE TARGETS IN DESI DIRECTLY!

distmu = skymap[sorter_i]["DISTMU"]
distsigma = skymap[sorter_i]["DISTSIGMA"]
distnorm = skymap[sorter_i]["DISTNORM"]


In [ ]:
# len(UNIQ), len(data), len(skymap)

In [ ]:
#data = data.keep_columns(["TARGETID", "TARGET_RA", "TARGET_DEC", "TILEID", "NIGHT", "Z", "ZERR", "ZWARN", "DELTACHI2", "FLUX_Z", "BGS_TARGET", "EBV", "SERSIC", "MWS_TARGET", "FILENAME"])
                          

In [ ]:
data = hstack([data, Table({"UNIQ": uniq}), Table({"PROBDENSITY": probdensity}), Table({"DISTMU": distmu}), Table({"DISTSIGMA": distsigma}), Table({"DISTNORM": distnorm})])

In [ ]:
data[:5]

In [ ]:
print(f"The highest and lowest probability values in our catalog are: {np.min(probdensity)} and {np.max(probdensity)}")

### Sort the skymap table and get 90% credible region

In [ ]:
# copy the table
skymap_sort = Table(skymap, copy = True)

In [ ]:
# check whether copy worked:
# skymap_sort, skymap
# np.max(skymap["UNIQ"])

In [ ]:
# now sort the skymap by its probability (descending)
skymap_sort.sort('PROBDENSITY', reverse=True)

In [ ]:
# this shows the first entries sorted be their probability
skymap_sort[:5]

In [ ]:
# now get 90% credible region area

level, ipix = ah.uniq_to_level_ipix(skymap_sort['UNIQ'])
pixel_area = ah.nside_to_pixel_area(ah.level_to_nside(level)) # this is the area each pixel contains

# calculate the probability per pixel (careful: since the pixels differ in size, this is not neceseraliy a useful quantity
prob = pixel_area * skymap_sort['PROBDENSITY']

# calc cumsum of probabilities and get index of pixel that sums up to 0.9
cumprob = np.cumsum(prob)
i = cumprob.searchsorted(0.9)

# print area
area_90 = pixel_area[:i].sum()
area_90.to_value(u.deg**2)

### can be deleted, just tried things out

In [ ]:
skymap_sort = hstack([skymap_sort, Table({"PROB": prob})])

In [ ]:
np.max(prob)

In [ ]:
skymap_sort[:5]

In [ ]:
skymap_prob = Table(skymap_sort, copy = True)

In [ ]:
skymap_prob.sort("PROB", reverse = True)

In [ ]:
skymap_prob[:5]

### Compare Tables, only leave 90% credible region in DESI data table

Now we can get a Table that only contains the 90% credible region. From there we only leave the matching objects in the DESI data

In [ ]:
# we simply select only the entries that are within the 90% credible region
skymap_90 = skymap_sort[:i]

In [ ]:
print("This is the number of UNIQ ID of objects which should lie within the 90% credible region (NOT THE NUMBER OF OBJECTS, as there can be multiple objects per UNIQ ID):", len(np.intersect1d(skymap_90["UNIQ"], data["UNIQ"])))

In [ ]:
mask = np.in1d(data["UNIQ"], skymap_90["UNIQ"])
data_90 = data[mask]

In [ ]:
print(f"We now have {len(data_90)} objects in the 90% credible region")
print("Check if the region from where the galaxies have been selected makes any sense (RA; DEC): ", np.min(data_90["TARGET_RA"]), np.max(data_90["TARGET_RA"]), np.min(data_90["TARGET_DEC"]), np.max(data_90["TARGET_DEC"]))

max_probdens_in_desi = np.max(data_90["PROBDENSITY"])
max_probdens_in_ligo = np.max(skymap["PROBDENSITY"]).to_value(u.deg**-2)
print(f"We can also check whether we have at least one galaxy in the UNIQ pixel with the highest probability from LIGO...")
print(f"This is {np.allclose(max_probdens_in_desi, max_probdens_in_ligo)}")

In [ ]:
data_90.sort('PROBDENSITY', reverse=True)

In [ ]:
data_90[:2]

In [ ]:
print("We have now performed all the necessary steps to get the localization selection in 2D, i.e. the angle on the sky")
min_z = np.min(data_90["Z"])
max_z = np.max(data_90["Z"])
print(f"However, we have not used the distance data at all. Therefore our redhsifts range from {min_z} to {max_z}")

### Calculate all the distances of the objects from the redshifts

In [ ]:
distances = Distance(z=data_90["Z"], cosmology=Planck18)
distances_err = Distance(z=data_90["ZERR"], cosmology=Planck18)

data_90 = hstack([data_90, Table({"DIST_Z": distances}), Table({"DIST_Z_ERR": distances_err})])

In [ ]:
data_90[:5]

### Lets check how much the distance measurement changes in the 90% credible region of LIGO and compare to our DESI data

We should see a lot more data in DESI (since we haven't done any redshift selection and there can be multiple objects per GW-pixel). Be carfeul, we are comparing actual object counts with the number of pixels.


We still see, that most of the GW localization is roughly between 780Mpc and 950Mpc. DESI definitely has a most of its data in this range as well.

In [ ]:
dist_max_90 = np.max(skymap_90["DISTMU"])*u.Mpc
dist_min_90 = np.min(skymap_90["DISTMU"])*u.Mpc
print(f"Our distance ranges from {dist_min_90} to {dist_max_90}")
print("Let's visualize the distance distribution of the LIGO event in the 90% credible region")

fig, ax = plt.subplots(ncols = 3, figsize = (15,8))


ax[0].hist(data_90["DIST_Z"], bins = "auto", color = "orange")
ax[1].hist(data_90["DIST_Z"], bins = "auto", color = "green")
ax[2].hist(skymap_90["DISTMU"], bins = "auto", color = "blue")


for axis in ax:
    axis.set_xlabel("distance bin (Mpc)")

ax[0].set_ylabel("object count")
ax[1].set_ylabel("object count")
ax[2].set_ylabel("pixel count")

ax[1].set_xlim(0,1000)
    
plt.show()

In [ ]:
len(skymap_90), len(data_90)

### To reduce our data even more, lets cut out the unreasonably far away objects

We simply take maximum and minimum distance we get from the LIGO skymap (i.e. DISTMU+-DISTERR) and check, whether every galaxy in that pixel falls inside this range.

In [ ]:
z_selection_mask = data_90["DIST_Z"] - data_90["DIST_Z_ERR"] <  data_90["DISTMU"] + data_90["DISTSIGMA"]
z_selection_mask &= data_90["DIST_Z"] + data_90["DIST_Z_ERR"] >  data_90["DISTMU"] - data_90["DISTSIGMA"]

data_90_z = data_90[z_selection_mask]

In [ ]:
remaining_obj = np.size(np.where(z_selection_mask == True))
deleted_obj = np.size(np.where(z_selection_mask == False))

print(f"The z-range selection has deleted {deleted_obj} objects from originally {deleted_obj+remaining_obj} in the data set")
print(f"Therefore, we have {remaining_obj} objects left in the search region")

In [ ]:
highest_prob_dens = np.max(data_90_z["PROBDENSITY"])
print(f"Our highest remaing probability density is {highest_prob_dens} compared to the originally highest prob density of {max_probdens_in_ligo}")
print(f"The difference in prob density is {max_probdens_in_ligo-highest_prob_dens}")

In [ ]:
data_90_z[:5]

### get the apparent, absolute magnitudes and luminosities

in order to do this: flux > 0 selection

In [ ]:
# get the apparent mags
data_90_z = data_90_z[data_90_z["FLUX_Z"] > 0]
app_mag_z = gs.app_mag(data_90_z["FLUX_Z"])
flux_selection_cut = len(data_90_z)
print(f"We have {flux_selection_cut} objects remaining in our catalog, i.e. {remaining_obj-flux_selection_cut} had to be cut out due to bad fluxes")

# get the absolute mags
abs_mag_z = gs.abs_mag(app_mag_z, data_90_z["Z"], 0,0,0)

# get the luminosities
lums_z = gs.lum_z(abs_mag_z)

In [ ]:
data_f = hstack([data_90_z, Table({"APP_MAG_Z": app_mag_z}), Table({"ABS_MAG_Z": abs_mag_z}), Table({"LUM_Z": lums_z})])

In [ ]:
data_f[:5]

In [ ]:
max_lum_z = np.max(data_f["LUM_Z"])*u.W
prob_dens_max_lum = data_f[data_f["LUM_Z"] == max_lum_z]["PROBDENSITY"]
print(f"The maximum luminosity in our cataloge in the z-band is {max_lum_z} with a prob density of {prob_dens_max_lum}")

In [ ]:
arg_errfc = abs(data_f["DIST_Z"]-data_f["DISTMU"])/(data_f["DIST_Z_ERR"]**2+data_f["DISTSIGMA"]**2)
P_gal_unnorm = data_f["LUM_Z"]*data_f["PROBDENSITY"]*(1-sc.special.erf(arg_errfc))
#dist_mean*u.Mpc, dist_std*u.Mpc
arg_errfc_c = abs(data_f["DIST_Z"].value-dist_mean)/(data_f["DIST_Z_ERR"].value**2+dist_std**2)
P_gal_unnorm_const = data_f["LUM_Z"]*data_f["PROBDENSITY"]*(1-sc.special.erf(arg_errfc_c))

# now lets quickly normalize

P_gal_sum = np.sum(P_gal_unnorm)
P_gal_const_sum = np.sum(P_gal_unnorm_const)

P_gal = 1/P_gal_sum * P_gal_unnorm
P_gal_const = 1/P_gal_const_sum * P_gal_unnorm_const

In [ ]:
data_f = hstack([data_f, Table({"P_GAL": P_gal.value}), Table({"P_GAL_C": P_gal_const.value})])

In [ ]:
data_f[:5]

## Analyse the data: How much luminosity do we cover? Which galaxies should we observe?

Lets look into what information we can retrieve from here. First we define, how many galaxies we could observe (N_galaxies).

We then calculate the luminosity (lum) from all the galaxies in this area, i.e. the total luminosity. From there do some comparison between the covered luminosity and total luminosity depending on the ranking by probability:
- simple "Maximum lum that could be covered" by looking at the brightest galaxies
- Calculate the probability this way (from https://arxiv.org/pdf/1710.05452.pdf w/o normalization), both using a static and variable distance and error: $$P_{gal} = k^{-1}\tilde{L_z}\cdot P_{2D}\left(1-\text{erf}\left(\frac{|D_{Gal}-D_{LVC}|}{\sigma_{D,gal}^{2}+\sigma_{D,LVC}^{2}}\right)\right)$$
- others may follow...

Since we have the actual luminosity (here in z-band though), we don't need to again calculate the $$\tilde{L}_{gal}$$ values. In the mentioned paper, they only use a fixed distance and error for the LIGO data. I will try both, to get an idea what the differences are.

In [ ]:
N_galaxies = 20 # how many galaxies we can cover

In [ ]:
total_lum = np.sum(data_f["LUM_Z"])

### Compare lum with luminosity ranking

https://iopscience.iop.org/article/10.3847/0067-0049/226/1/10

In [ ]:
data_sort_by_lums = data_f.group_by("LUM_Z")
data_sort_by_lums = data_sort_by_lums[::-1]

In [ ]:
lum_after_N_highest = np.sum(data_sort_by_lums["LUM_Z"][:N_galaxies])

In [ ]:
print(f"The theoretical maximum luminosity we could cover is {lum_after_N_highest}, i.e. {lum_after_N_highest*100/total_lum} % of the total luminosity")
print("The TARGETID values of these galaxies are: \n", data_sort_by_lums["TARGETID"][:N_galaxies])

### Compare lum with P_Gal ranking

In [ ]:
data_sort_by_PGal = data_f.group_by("P_GAL")
data_sort_by_PGal = data_sort_by_PGal[::-1]

lum_after_N_PGal = np.sum(data_sort_by_PGal["LUM_Z"][:N_galaxies])

In [ ]:
print(f"The maximum luminosity we cover when ranked by P_gal is {lum_after_N_PGal}, i.e. {lum_after_N_PGal*100/total_lum}% of the total luminosity") 
print("The TARGETID values of these galaxies are: \n", data_sort_by_PGal["TARGET_RA", "TARGET_DEC", "SERSIC", "TARGETID", "P_GAL"][:N_galaxies])

In [ ]:
data_sort_by_PGal_C = data_f.group_by("P_GAL_C")
data_sort_by_PGal_C = data_sort_by_PGal_C[::-1]

lum_after_N_PGal_C = np.sum(data_sort_by_PGal_C["LUM_Z"][:N_galaxies])

In [ ]:
print(f"The maximum luminosity we cover when ranked by P_gal_C is {lum_after_N_PGal_C}, i.e. {lum_after_N_PGal_C*100/total_lum}% of the total luminosity") 
print("The TARGETID values of these galaxies are: \n", data_sort_by_PGal_C["TARGETID", "P_GAL"][:N_galaxies])

In [ ]:
data_sort_by_PGal["TARGET_RA", "TARGET_DEC", "SERSIC", "TARGETID", "P_GAL"][:N_galaxies].write('PGAL_S200129.ecsv', delimiter=',', format='ascii', overwrite = True)